# Simulation Examples
In this notebook I want to keep a log of interesting simulation examples that I find. Interesting means that they exhibit some behaviour that is e.g. not straightforward to understand. Also I would like to understand in which cases the AL approximation outperforms the UT.

## Preamble

In [3]:
from google.colab import drive
drive.mount('/content/drive')
%cd drive/MyDrive/python/BayesianFiltering
from codebase.utils import split_to_sigma_points
import numpy as np
import copy
import matplotlib.pyplot as plt
import pandas as pd

Mounted at /content/drive
/content/drive/MyDrive/python/BayesianFiltering


## Experiment 1


In [4]:
g = lambda D, Pv, L, Nv, H: (2*L**2/Nv) * np.trace(Pv-D) + (1 / 4) * np.trace(np.matmul(D, H))**2

matrix_projection = lambda A, B : ( np.trace(np.matmul(A.transpose(),B)) / np.trace(np.matmul(B.transpose(),B))) * B

def project_to_PSD(Delta):
  evals, evec = np.linalg.eig(Delta)
  nonzero_eig = np.sum(evals>0)
  new_evals = np.multiply(evals>0, evals)
  new_Delta = np.matmul(np.matmul(evec, np.diag(new_evals)), evec.transpose())
  return (new_Delta + new_Delta.transpose())/2

def gradient_descent(dim, N, L, X0, P, H, Nsteps, eta):
  X = X0
  for i in range(Nsteps):
    X = X - eta * (-(2*L**2 / N) * np.eye(dim) + (1 / 2) * np.trace(np.matmul(H, X)) * H)
  return X

def SPD_opt(dim, N, L, X0, P, H, Nsteps, eta):
  X = X0 
  for i in range(Nsteps):
    X = gradient_descent(dim, N, L, X, P, H, 1, eta**i)
    X = project_to_PSD(X)
    X = P - project_to_PSD(P-X)
    X = project_to_PSD(X)
  return X

In [8]:
# Fixed parameters
dx = 10
dy = 1
Q = 1 
N = 1000000
Nsim = 100
Lipschitz = 1

# ## Choice of m and P
m = np.ones(dx) # np.random.multivariate_normal(np.ones(dx), 0.0001 * np.eye(dx)) 
CholP = np.random.rand(dx, dx)
P =  np.eye(dx) #np.dot(CholP, CholP.transpose())  # 

# Definition of non-linearity
## 1  (dx = any)
# p = -3
# f = lambda x: (1 + np.dot(x,x))**(p/2)
# J = lambda x : p * (1 + np.dot(x,x))**(p/2-1) * x
# H = lambda x : 2 * p*(p/2-1)*(1 + np.dot(x,x))**(p/2-2)*np.outer(x,x) + np.eye(dx)*p* (1 + np.dot(x,x))**(p/2-1)

## 2 Sinc (dx = any)
f = lambda x: np.sin(np.dot(x,x)) / np.dot(x,x) if np.dot(x,x)>0 else 1
J = lambda x: 2 * (np.dot(x,x) * np.cos(np.dot(x,x)) - np.sin(np.dot(x,x))) / (np.dot(x,x))**2 * x
H = lambda x: -4*(np.sin(np.dot(x,x))/np.dot(x,x) + \
                  2*(np.cos(np.dot(x,x))*np.dot(x,x) - \
                     np.sin(np.dot(x,x))) / np.dot(x,x)**3 ) * \
                     np.outer(x,x) + 2 * (np.dot(x,x)*np.cos(np.dot(x,x)) - \
                                          np.sin(np.dot(x,x))) / \
                                          (np.dot(x,x))**2 * np.eye(dx)

# 3 Linear-Nonlinear product  (dx = 2)
# a = 200
# f = lambda x: a * x[0] * np.sin(x[1])
# J = lambda x: a * np.array([np.sin(x[1]), x[0]*np.cos(x[1])])
# H = lambda x: a * np.array([[0, np.cos(x[1])],[np.cos(x[1]), -x[0]*np.sin(x[1])]])

## 4 Linear-Nonlinear sum (dx = 2)
# f = lambda x: x[0] + np.sin(x[1])
# J = lambda x: np.array([1, np.cos(x[1])])
# H = lambda x: np.array([[0,0],[0, -np.sin(x[1])]])

## 5 Quadratic (dx = 2)
# A = np.array([[2, 0 ],[0, -1]])
# f = lambda x: np.dot(x, np.matmul(A,x))/2
# J = lambda x: np.matmul(A, x)
# H = lambda x: A

## 5 Quadratic (dx = whatever, A = positive definite)
# CholA = np.random.rand(dx, dx)
# A = np.dot(CholA, CholA.transpose()) 
# f = lambda x: np.dot(x, np.matmul(A,x))/2
# J = lambda x: np.matmul(A, x)
# H = lambda x: A


# Error Array Initialization

RSE_mean_MC100 = np.zeros(Nsim)
RSE_var_MC100  = np.zeros(Nsim)
RSE_cov_MC100 = np.zeros(Nsim)
RSE_mean_L  = np.zeros(Nsim)
RSE_var_L  = np.zeros(Nsim)
RSE_cov_L = np.zeros(Nsim)
RSE_mean_U = np.zeros(Nsim)
RSE_var_U = np.zeros(Nsim)
RSE_cov_U = np.zeros(Nsim)
RSE_mean_AL1 = np.zeros(Nsim)
RSE_var_AL1 = np.zeros(Nsim)
RSE_cov_AL1 = np.zeros(Nsim)
RSE_mean_AL2 = np.zeros(Nsim)
RSE_var_AL2  = np.zeros(Nsim)
RSE_cov_AL2 = np.zeros(Nsim)


est_mean_MC = np.zeros(Nsim)
est_var_MC  = np.zeros(Nsim)
est_mean_MC100 = np.zeros(Nsim)
est_var_MC100  = np.zeros(Nsim)
est_mean_L = np.zeros(Nsim)
est_var_L  = np.zeros(Nsim)
est_mean_U = np.zeros(Nsim)
est_var_U  = np.zeros(Nsim)
est_mean_AL1 = np.zeros(Nsim)
est_var_AL1  = np.zeros(Nsim)
est_mean_AL2 = np.zeros(Nsim)
est_var_AL2  = np.zeros(Nsim)


## Diagnostic Dataframes Initialization ########################################

ALdiagnostic_df = pd.DataFrame(columns = ['run {}'.format(i) for i in range(Nsim)])

## Baseline Estimate  ##########################################################

if dx == 1:
  x = np.random.normal(m, P, size = N)
else:
  x = np.random.multivariate_normal(m, P, size = N)
y = np.array(list(map(f, x))) + np.random.normal(0, Q, N)

meanY_MC = np.mean(y)
varY_MC = np.var(y) 
covXY_MC = np.matmul(np.transpose(x-m),y-np.mean(y))/(N-1)



## Simulation Begins 

for i in range(Nsim):

  ## MC M
  M = 10
  if dx == 1:
    x_MC100 = np.random.normal(m, P, size = M)
  else:
    x_MC100 = np.random.multivariate_normal(m, P, size = M)
  y_MC100 = np.array(list(map(f, x_MC100)))
  meanY_MC100 = np.mean(y_MC100)
  varY_MC100 = np.var(y_MC100) + Q
  covXY_MC100 = np.matmul(np.transpose(x_MC100-m), y_MC100-meanY_MC100)/(M-1)

  ## Taylor ####################################################################
  meanY_L = f(m) + (1/2) * np.trace(np.matmul(H(m), P))
  if dx == 1:
    varY_L = Q + J(m) * P *J(m) 
    covXY_L = P * J(m)
  else:
    varY_L = Q + np.dot(J(m), np.matmul(P,J(m))) + (1/2) * np.trace(np.matmul(np.matmul(H(m), P), np.matmul(H(m), P)))
    covXY_L = np.matmul(P, J(m))

  ## Unscented #################################################################
  alpha = 1e-3
  beta = 2
  lam = (alpha**2-1)*dx
  sigma_points = split_to_sigma_points(m, P, lam)
  Y_sigma_points = np.array(list(map(f, sigma_points)))

  meanY_U = (lam / (dx+lam)) * Y_sigma_points[0] + 1 / (2*(dx+lam)) * sum(Y_sigma_points[1:])
  varY_U = Q + (lam/(dx+lam) + 1 - alpha**2 + beta) * (Y_sigma_points[0]-meanY_U)**2 + \
                            1/(2*(dx+lam)) * sum((Y_sigma_points[1:]-meanY_U)**2)

  covXY_U = (lam/(dx+lam) + 1 - alpha**2 + beta) * (Y_sigma_points[0]-meanY_U) * (sigma_points[0] - m) + \
                            1/(2*(dx+lam))* np.matmul((Y_sigma_points[1:]-meanY_U), (sigma_points[1:] - m))
                        
  ## MCLA ######################################################################
  Nz = M

  # AL1 (first variety)##########################

  # Projection 
  if abs(np.trace(H(m))) > 0.01:
    Delta = P - matrix_projection(P, H(m))
  else:
    Delta = P

  mz = m
  Pz = P-Delta

  # # MC approx of z-integral
  z = np.random.multivariate_normal(mz, Pz, Nz)
  x_tilde = z
  y_tilde = np.array(list(map(f, x_tilde)))
  meanY_AL1 = np.mean(y_tilde)
  gradients = np.array(list(map(J, x_tilde)))
  varY_AL1 = Q + sum((y_tilde-meanY_AL1)**2) / Nz + np.trace(np.matmul(np.matmul(gradients, Delta), np.transpose(gradients)))/Nz
  covXY_AL1 = np.matmul(Delta, np.mean(gradients, axis =0)) + np.matmul(np.transpose(x_tilde - m), y_tilde - meanY_AL1) / Nz

  # AL2 (second variety) ##########################

  # Projected Gradient Descent 
  Delta = SPD_opt(dx, M, Lipschitz, P, P, H(m), 100, 0.01)
  mz = m
  Pz = P-Delta

  # # MC approx of z-integral
  z = np.random.multivariate_normal(mz, Pz, Nz)
  x_tilde = z
  y_tilde = np.array(list(map(f, x_tilde)))
  meanY_AL2 = np.mean(y_tilde)
  gradients = np.array(list(map(J, x_tilde)))
  varY_AL2 = Q + sum((y_tilde-meanY_AL2)**2) / Nz + np.trace(np.matmul(np.matmul(gradients, Delta), np.transpose(gradients)))/Nz
  covXY_AL2 = np.matmul(Delta, np.mean(gradients, axis =0)) + np.matmul(np.transpose(x_tilde - m), y_tilde - meanY_AL2) / Nz

  ############## Diagnostics Dataframe ############################

  ALdiagnostic_df['run ' + str(i)] = np.reshape(z, [M*dx, ])

  ############## Computation of errors #######################

  RSE_mean_MC100[i] = (meanY_MC100 - meanY_MC)**2 / np.abs(meanY_MC)
  RSE_var_MC100[i]  = (varY_MC100 - varY_MC)**2 / np.abs(varY_MC)
  RSE_cov_MC100[i]  = np.dot(covXY_MC100 - covXY_MC, covXY_MC100 - covXY_MC) / np.dot(covXY_MC, covXY_MC) / dx
  RSE_mean_L[i] = (meanY_L-meanY_MC)**2 / np.abs(meanY_MC)
  RSE_var_L[i]  = (varY_L - varY_MC)**2 / np.abs(varY_MC)
  RSE_cov_L[i]  = np.dot(covXY_L - covXY_MC, covXY_L - covXY_MC) / np.dot(covXY_MC, covXY_MC) / dx
  RSE_mean_U[i] = (meanY_U-meanY_MC)**2 / np.abs(meanY_MC)
  RSE_var_U[i]  = (varY_U - varY_MC)**2 / np.abs(varY_MC)
  RSE_cov_U[i]  = np.dot(covXY_U - covXY_MC, covXY_U - covXY_MC) / np.dot(covXY_MC, covXY_MC) / dx
  RSE_mean_AL1[i] = (meanY_AL1-meanY_MC)**2 / np.abs(meanY_MC)
  RSE_var_AL1[i]  = (varY_AL1 - varY_MC)**2 / np.abs(varY_MC)
  RSE_cov_AL1[i]  = np.dot(covXY_AL1 - covXY_MC, covXY_AL1 - covXY_MC) / np.dot(covXY_MC, covXY_MC) / dx
  RSE_mean_AL2[i] = (meanY_AL2 - meanY_MC)**2 / np.abs(meanY_MC)
  RSE_var_AL2[i]  = (varY_AL2 - varY_MC)**2 / np.abs(varY_MC)
  RSE_cov_AL2[i]  = np.dot(covXY_AL2 - covXY_MC, covXY_AL2 - covXY_MC) / np.dot(covXY_MC, covXY_MC) / dx


  est_mean_MC[i] = meanY_MC
  est_var_MC[i] = varY_MC
  est_mean_MC100[i] = meanY_MC100
  est_var_MC100[i] = varY_MC100
  est_mean_L[i] = meanY_L
  est_var_L[i]  = varY_L
  est_mean_U[i] = meanY_U
  est_var_U[i]  = varY_U
  est_mean_AL1[i] = meanY_AL1
  est_var_AL1[i]  = varY_AL1
  est_mean_AL2[i] = meanY_AL2
  est_var_AL2[i]  = varY_AL2  

RRMSE_mean_MC100  = np.sqrt(np.mean(RSE_mean_MC100))
RRMSE_mean_L  = np.sqrt(np.mean(RSE_mean_L))
RRMSE_mean_U  = np.sqrt(np.mean(RSE_mean_U))
RRMSE_mean_AL1  = np.sqrt(np.mean(RSE_mean_AL1))
RRMSE_mean_AL2  = np.sqrt(np.mean(RSE_mean_AL2))
RRMSE_var_MC100  = np.sqrt(np.mean(RSE_var_MC100))
RRMSE_var_L  = np.sqrt(np.mean(RSE_var_L))
RRMSE_var_U  = np.sqrt(np.mean(RSE_var_U))
RRMSE_var_AL1  = np.sqrt(np.mean(RSE_var_AL1))
RRMSE_var_AL2  = np.sqrt(np.mean(RSE_var_AL2))
RRMSE_cov_MC100  = np.sqrt(np.mean(RSE_cov_MC100))
RRMSE_cov_L  = np.sqrt(np.mean(RSE_cov_L))
RRMSE_cov_U  = np.sqrt(np.mean(RSE_cov_U))
RRMSE_cov_AL1  = np.sqrt(np.mean(RSE_cov_AL1))
RRMSE_cov_AL2  = np.sqrt(np.mean(RSE_cov_AL2))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:160: RuntimeWarning: covariance is not positive-semidefinite.


In [9]:
df = pd.DataFrame(columns = [' ','MC100','Taylor','Unscented','AL1','AL2'])
df[' '] = ['mean RRMSE', 'varY RRMSE', 'covXY RRMSE']
df['MC100'] = [RRMSE_mean_MC100, RRMSE_var_MC100, RRMSE_cov_MC100]
df['Taylor'] = [RRMSE_mean_L, RRMSE_var_L, RRMSE_cov_L]
df['Unscented'] = [RRMSE_mean_U, RRMSE_var_U, RRMSE_cov_U]
df['AL1'] = [RRMSE_mean_AL1, RRMSE_var_AL1, RRMSE_cov_AL1]
df['AL2'] = [RRMSE_mean_AL2, RRMSE_var_AL2, RRMSE_cov_AL2]
#print(df.to_latex(index=False))  
df

,,MC100,Taylor,Unscented,AL1,AL2
0,mean RRMSE,0.506437,19.181563,19.181931,1.156083,0.836897
1,varY RRMSE,0.001529,3.857603,1.006986,0.142807,0.039746
2,covXY RRMSE,5.647556,63.029471,63.028572,12.315252,7.137987


### Diagnostics

In [7]:
df_est = pd.DataFrame(columns = ['mean MC1e5', 'var MC1e5','mean MC100', 'var MC100',
                                 'mean Taylor', 'var Taylor',
                                 'mean Unscented', 'var Unscented', 'mean AL1', 
                                 'var AL1', 'mean AL2', 'var AL2'])
df_est['mean MC1e5'] =  est_mean_MC
df_est['var MC1e5']  = est_var_MC
df_est['mean MC100'] =  est_mean_MC100
df_est['var MC100']  = est_var_MC100
df_est['mean Taylor'] = est_mean_L 
df_est['var Taylor'] = est_var_L
df_est['mean Unscented'] = est_mean_U
df_est['var Unscented'] = est_var_U
df_est['mean AL1'] = est_mean_AL1
df_est['var AL1']= est_var_AL1
df_est['mean AL2'] = est_mean_AL2
df_est['var AL2']= est_var_AL2
df_est

,mean MC1e5,var MC1e5,mean MC100,var MC100,mean Taylor,var Taylor,mean Unscented,var Unscented,mean AL1,var AL1,mean AL2,var AL2
0,0.012691,1.002872,0.014334,1.000062,0.007022,1.001162,0.007022,1.00139,0.027457,1.001391,0.025482,1.000633
1,0.012691,1.002872,0.013096,1.000092,0.007022,1.001162,0.007022,1.00139,0.032366,1.003465,0.023756,1.000422
2,0.012691,1.002872,0.010036,1.000058,0.007022,1.001162,0.007022,1.00139,0.035338,1.004549,0.019507,1.000399
3,0.012691,1.002872,0.009457,1.000028,0.007022,1.001162,0.007022,1.00139,0.022841,1.001222,0.022869,1.000306
4,0.012691,1.002872,0.008556,1.000012,0.007022,1.001162,0.007022,1.00139,0.036214,1.003600,0.019642,1.000318
...,...,...,...,...,...,...,...,...,...,...,...,...
95,0.012691,1.002872,0.015603,1.000202,0.007022,1.001162,0.007022,1.00139,0.018648,1.000780,0.020894,1.000396
96,0.012691,1.002872,0.012301,1.000020,0.007022,1.001162,0.007022,1.00139,0.027670,1.001603,0.018942,1.000262
97,0.012691,1.002872,0.016138,1.000089,0.007022,1.001162,0.007022,1.00139,0.028124,1.002162,0.024239,1.000393
98,0.012691,1.002872,0.014431,1.000145,0.007022,1.001162,0.007022,1.00139,0.020840,1.000897,0.024751,1.000435


In [ ]:
run = 2
z_array = np.reshape( np.array(ALdiagnostic_df['run ' + str(run)]), [M, dx])
y_array = np.array(list(map(f, z_array)))
print(z_array)
print(y_array)
np.mean(y_array)

[[ 1.63141766  1.22565513]
 [ 0.44746318  2.58863066]
 [ 0.29784892 -1.60172862]
 [ 1.96882444  1.40647177]
 [ 1.3704786   0.99932897]
 [ 1.17489213  0.23273214]
 [ 0.99440149  0.80385503]
 [ 1.63458285  1.15787215]
 [ 0.53324366  2.00327967]
 [ 0.46845303 -1.28262639]]
[307.04183885  47.00243628 -59.54128848 388.4605091  230.54416744
  54.1946867  143.20108896 299.43965593  96.82933975 -89.82732755]


141.73451069892042

In [ ]:
print('Delta=', Delta, '\n eigs = ', np.linalg.eig(Delta)[0])
print('Pz=',Pz, '\n eigs = ', np.linalg.eig(Pz)[0])

Delta= [[0.20807909 0.        ]
 [0.         0.99899076]] 
 eigs =  [0.20807909 0.99899076]
Pz= [[0.79192091 0.        ]
 [0.         0.00100924]] 
 eigs =  [0.79192091 0.00100924]


In [ ]:
Delta = (4/3) * np.array([[1/2, 0],[0, 1]])
Pz = P-Delta
print('Pz=',Pz, '\n eigs = ', np.linalg.eig(Pz)[0])
Delta = P - project_to_PSD(P-Delta)
Pz = P-Delta
print('Pz=',Pz, '\n eigs = ', np.linalg.eig(Pz)[0])
Delta

Pz= [[ 0.33333333  0.        ]
 [ 0.         -0.33333333]] 
 eigs =  [ 0.33333333 -0.33333333]
Pz= [[0.33333333 0.        ]
 [0.         0.        ]] 
 eigs =  [0.33333333 0.        ]


array([[0.66666667, 0.        ],
       [0.        , 1.        ]])

## Comments

Experiments for which the MCLA method gives better result than MC.

### Case 1
At the time of writing the AL1 is implemented with a projection of P on H_perp and AL2 same as AL1 followed by a projection of PSD cone.

|index| |MC100|Taylor|Unscented|AL1|AL2|
|---|---|---|---|---|---|---|
|0|mean RRMSE|0\.2147506228851694|0\.9846019520535367|0\.9846008324674256|0\.05828208508954865|0\.05927688494376014|
|1|varY RRMSE|0\.3542487070225271|3\.53374924346583|3\.5337447169519876|0\.0655453486979092|0\.11026733767100663|
|2|covXY RRMSE|0\.11521607142822617|0\.3062193289097994|0\.30621888765173855|0\.01953128634986099|0\.021551038035518576|

```
# Fixed parameters
dx = 2
dy = 1
Q = 1 
N = 100000
Nsim = 100
# Choice of m and P
m = np.random.multivariate_normal(np.ones(dx), 0.1 * np.eye(dx)) 
CholP = np.random.rand(dx, dx)
P = 10*np.dot(CholP, CholP.transpose())  # 1 * np.eye(dx) #
## 4 Linear-Nonlinear sum (dx = 2)
f = lambda x: x[0] + np.sin(x[1])
J = lambda x: np.array([1, np.cos(x[1])])
H = lambda x: np.array([[0,0],[0, -np.sin(x[1])]])
```

### Case 2
At the time of writing the AL1 is implemented with a projection of P on H_perp and AL2 uses two projections on the PSD cones.

|index| |MC100|Taylor|Unscented|AL1|AL2|
|---|---|---|---|---|---|---|
|0|mean RRMSE|1\.6511804515692747|0\.0016366495190621672|0\.0016366495518205996|0\.5262063428243483|0\.44088999566754883|
|1|varY RRMSE|4\.237197520604308|0\.0010235932206890195|4\.715063881321998|3\.378873630667752|3\.5357997323474892|
|2|covXY RRMSE|1\.2140410858583208|0\.002546661739549881|0\.0025466617395792945|0\.38951996191744487|0\.2465948515788907|

```
# Fixed parameters
dx = 2
dy = 1
Q = 1 
N = 1000000
Nsim = 100

# ## Choice of m and P
m = np.random.multivariate_normal(np.ones(dx), 0.1 * np.eye(dx)) 
CholP = np.random.rand(dx, dx)
P = 10*np.dot(CholP, CholP.transpose())  # 1 * np.eye(dx) #

# Definition of non-linearity
## 5 Quadratic (dx = 2)
A = np.array([[2, 0 ],[0, -1]])
f = lambda x: np.dot(x, np.matmul(A,x))/2
J = lambda x: np.matmul(A, x)
H = lambda x: A
```





## Experiment 2 (in 1D)

In [ ]:
# Fixed parameters
dx = 1
dy = 1
Q = 1 
N = 1000000
Nsim = 100
o = 0.2
k=4
# ## Choice of m and P
m =  0.0000001
P = 0.5

# Definition of non-linearity
## 1  
# f = lambda x : np.sin(o * x ** k)
# J = lambda x : o * np.cos(o * x ** k) * k * x ** (k-1)
# H = lambda x : -o **2 * np.sin(o * x ** k ) * k * x ** (k-1) + k * (k-1) * x ** (k-2) * o * np.cos(o * x ** k)

## 2
f = lambda x : 1 / x
J = lambda x :  - 1 / (x**2)
H = lambda x : 2 / (x**3)



# Error arrays
RSE_mean_MC100 = np.zeros(Nsim)
RSE_var_MC100  = np.zeros(Nsim)
RSE_cov_MC100 = np.zeros(Nsim)
RSE_mean_L  = np.zeros(Nsim)
RSE_var_L  = np.zeros(Nsim)
RSE_cov_L = np.zeros(Nsim)
RSE_mean_U = np.zeros(Nsim)
RSE_var_U = np.zeros(Nsim)
RSE_cov_U = np.zeros(Nsim)
RSE_mean_AL1 = np.zeros(Nsim)
RSE_var_AL1 = np.zeros(Nsim)
RSE_cov_AL1 = np.zeros(Nsim)
RSE_mean_AL2 = np.zeros(Nsim)
RSE_var_AL2  = np.zeros(Nsim)
RSE_cov_AL2 = np.zeros(Nsim)


est_mean_MC = np.zeros(Nsim)
est_var_MC  = np.zeros(Nsim)
est_mean_MC100 = np.zeros(Nsim)
est_var_MC100  = np.zeros(Nsim)
est_mean_L = np.zeros(Nsim)
est_var_L  = np.zeros(Nsim)
est_mean_U = np.zeros(Nsim)
est_var_U  = np.zeros(Nsim)
est_mean_AL1 = np.zeros(Nsim)
est_var_AL1  = np.zeros(Nsim)
est_mean_AL2 = np.zeros(Nsim)
est_var_AL2  = np.zeros(Nsim)

## Baseline Estimate  ##########################################################
x = np.random.normal(m, P, size = N)
y = np.array(list(map(f, x))) + np.random.normal(0, Q, N)

meanY_MC = np.mean(y)
varY_MC = np.var(y) 
covXY_MC = np.dot(x-m,y-np.mean(y))/(N-1)

meanY_MC = 0

for i in range(Nsim):

  ## MC M
  M = 100
  x_MC100 = np.random.normal(m, P, size = M)
  y_MC100 = np.array(list(map(f, x_MC100)))
  meanY_MC100 = np.mean(y_MC100)
  varY_MC100 = np.var(y_MC100) + Q
  covXY_MC100 = np.dot(x_MC100-m, y_MC100-meanY_MC100) / (M-1)

  ## Taylor ####################################################################
  meanY_L = f(m) + (1/2) * H(m) * P 
  varY_L = Q + J(m) * P *J(m) + (1/2) * H(m)**2 * P**2 
  covXY_L = P * J(m)

  ## Unscented #################################################################
  # alpha = 1e-3
  # beta = 2
  # lam = (alpha**2-1)*dx
  # sigma_points = split_to_sigma_points([m], P, lam)
  # Y_sigma_points = np.array(list(map(f, sigma_points)))

  meanY_U = 0 # (lam / (dx+lam)) * Y_sigma_points[0] + 1 / (2*(dx+lam)) * sum(Y_sigma_points[1:])
  varY_U = 0 # Q + (lam/(dx+lam) + 1 - alpha**2 + beta) * (Y_sigma_points[0]-meanY_U)**2 + \
                            #1/(2*(dx+lam)) * sum((Y_sigma_points[1:]-meanY_U)**2)

  covXY_U = 0 #(lam/(dx+lam) + 1 - alpha**2 + beta) * (Y_sigma_points[0]-meanY_U) * (sigma_points[0] - m) + \
                            #1/(2*(dx+lam))* np.matmul((Y_sigma_points[1:]-meanY_U), (sigma_points[1:] - m))
                        
  ## Augmented #################################################################
  Nz = M

  # AL1 (first variety)##########################

  Delta1 = 0.0 * P

  mz1 = m
  Pz1 = P-Delta1

  # # MC approx of z-integral
  z1 = np.random.normal(mz1, Pz1, Nz)
  x_tilde1 = z1
  y_tilde1 = np.array(list(map(f, x_tilde1)))
  meanY_AL1 = np.mean(y_tilde1)
  gradients1 = np.array(list(map(J, x_tilde1)))
  varY_AL1 = Q + np.var(y_tilde1) + np.dot(gradients1, gradients1) * Delta1 / Nz
  covXY_AL1 = Delta1 * np.mean(gradients1) + np.dot(x_tilde1 - m, y_tilde1 - meanY_AL1) / (Nz-1)

  # AL2 (second variety) ##########################
  Delta0 =  o**2 / Nz / H(m)**2
  Delta = Delta0 if Delta0 < P else P

  mz = m
  Pz = P-Delta

  # # MC approx of z-integral
  z = np.random.normal(mz, Pz, Nz)
  x_tilde = z
  y_tilde = np.array(list(map(f, x_tilde)))
  meanY_AL2 = np.mean(y_tilde)
  gradients = np.array(list(map(J, x_tilde)))
  varY_AL2 = Q + np.var(y_tilde) + np.dot(gradients, gradients) * Delta / Nz
  covXY_AL2 = Delta * np.mean(gradients) + np.dot(x_tilde - m, y_tilde - meanY_AL2) / (Nz-1)

  ############## Computation of errors #######################

  RSE_mean_MC100[i] = (meanY_MC100 - meanY_MC)**2 #/ np.abs(meanY_MC)**2
  RSE_var_MC100[i]  = (varY_MC100 - varY_MC)**2 / np.abs(varY_MC)**2
  RSE_cov_MC100[i]  = np.dot(covXY_MC100 - covXY_MC, covXY_MC100 - covXY_MC) / np.dot(covXY_MC, covXY_MC) / dx
  RSE_mean_L[i] = (meanY_L-meanY_MC)**2 #/ np.abs(meanY_MC)**2
  RSE_var_L[i]  = (varY_L - varY_MC)**2 / np.abs(varY_MC)**2
  RSE_cov_L[i]  = np.dot(covXY_L - covXY_MC, covXY_L - covXY_MC) / np.dot(covXY_MC, covXY_MC) / dx
  RSE_mean_U[i] = (meanY_U-meanY_MC)**2 #/ np.abs(meanY_MC)**2
  RSE_var_U[i]  = (varY_U - varY_MC)**2 / np.abs(varY_MC)**2
  RSE_cov_U[i]  = np.dot(covXY_U - covXY_MC, covXY_U - covXY_MC) / np.dot(covXY_MC, covXY_MC) / dx
  RSE_mean_AL1[i] = (meanY_AL1-meanY_MC)**2 #/ np.abs(meanY_MC)**2
  RSE_var_AL1[i]  = (varY_AL1 - varY_MC)**2 / np.abs(varY_MC)**2
  RSE_cov_AL1[i]  = np.dot(covXY_AL1 - covXY_MC, covXY_AL1 - covXY_MC) / np.dot(covXY_MC, covXY_MC) / dx
  RSE_mean_AL2[i] = (meanY_AL2 - meanY_MC)**2 #/ np.abs(meanY_MC)**2
  RSE_var_AL2[i]  = (varY_AL2 - varY_MC)**2 / np.abs(varY_MC)**2
  RSE_cov_AL2[i]  = np.dot(covXY_AL2 - covXY_MC, covXY_AL2 - covXY_MC) / np.dot(covXY_MC, covXY_MC) / dx


  est_mean_MC[i] = meanY_MC
  est_var_MC[i] = varY_MC
  est_mean_MC100[i] = meanY_MC100
  est_var_MC100[i] = varY_MC100
  est_mean_L[i] = meanY_L
  est_var_L[i]  = varY_L
  est_mean_U[i] = meanY_U
  est_var_U[i]  = varY_U
  est_mean_AL1[i] = meanY_AL1
  est_var_AL1[i]  = varY_AL1
  est_mean_AL2[i] = meanY_AL2
  est_var_AL2[i]  = varY_AL2

RRMSE_mean_MC100  = np.sqrt(np.mean(RSE_mean_MC100))
RRMSE_mean_L  = np.sqrt(np.mean(RSE_mean_L))
RRMSE_mean_U  = np.sqrt(np.mean(RSE_mean_U))
RRMSE_mean_AL1  = np.sqrt(np.mean(RSE_mean_AL1))
RRMSE_mean_AL2  = np.sqrt(np.mean(RSE_mean_AL2))
RRMSE_var_MC100  = np.sqrt(np.mean(RSE_var_MC100))
RRMSE_var_L  = np.sqrt(np.mean(RSE_var_L))
RRMSE_var_U  = np.sqrt(np.mean(RSE_var_U))
RRMSE_var_AL1  = np.sqrt(np.mean(RSE_var_AL1))
RRMSE_var_AL2  = np.sqrt(np.mean(RSE_var_AL2))
RRMSE_cov_MC100  = np.sqrt(np.mean(RSE_cov_MC100))
RRMSE_cov_L  = np.sqrt(np.mean(RSE_cov_L))
RRMSE_cov_U  = np.sqrt(np.mean(RSE_cov_U))
RRMSE_cov_AL1  = np.sqrt(np.mean(RSE_cov_AL1))
RRMSE_cov_AL2  = np.sqrt(np.mean(RSE_cov_AL2))

In [ ]:
df = pd.DataFrame(columns = [' ','MC100','Taylor','Unscented','AL1','AL2'])
df[' '] = ['mean RRMSE', 'varY RRMSE', 'covXY RRMSE']
df['MC100'] = [RRMSE_mean_MC100, RRMSE_var_MC100, RRMSE_cov_MC100]
df['Taylor'] = [RRMSE_mean_L, RRMSE_var_L, RRMSE_cov_L]
df['Unscented'] = [RRMSE_mean_U, RRMSE_var_U, RRMSE_cov_U]
df['AL1'] = [RRMSE_mean_AL1, RRMSE_var_AL1, RRMSE_cov_AL1]
df['AL2'] = [RRMSE_mean_AL2, RRMSE_var_AL2, RRMSE_cov_AL2]
#print(df.to_latex(index=False))  
df

,,MC100,Taylor,Unscented,AL1,AL2
0,mean RRMSE,29.253046,5.000000e+20,0.0,49.551999,39.426973
1,varY RRMSE,0.993355,5.813716e+34,1.0,1.006265,0.987480
2,covXY RRMSE,0.654095,4.995132e+13,1.0,6.213522,0.806920


In [ ]:
df_est = pd.DataFrame(columns = ['mean MC1e5', 'var MC1e5','mean MC100', 'var MC100',
                                 'mean Taylor', 'var Taylor',
                                 'mean Unscented', 'var Unscented', 'mean AL1', 
                                 'var AL1', 'mean AL2', 'var AL2'])
df_est['mean MC1e5'] =  est_mean_MC
df_est['var MC1e5']  = est_var_MC
df_est['mean MC100'] =  est_mean_MC100
df_est['var MC100']  = est_var_MC100
df_est['mean Taylor'] = est_mean_L 
df_est['var Taylor'] = est_var_L
df_est['mean Unscented'] = est_mean_U
df_est['var Unscented'] = est_var_U
df_est['mean AL1'] = est_mean_AL1
df_est['var AL1']= est_var_AL1
df_est['mean AL2'] = est_mean_AL2
df_est['var AL2']= est_var_AL2
df_est

,mean MC1e5,var MC1e5,mean MC100,var MC100,mean Taylor,var Taylor,mean Unscented,var Unscented,mean AL1,var AL1,mean AL2,var AL2
0,0.0,8.600351e+06,-1.769251,3093.517687,5.000000e+20,5.000000e+41,0.0,0.0,-1.045208,4.046190e+02,-3.186218,3.984639e+02
1,0.0,8.600351e+06,-0.816811,1551.946712,5.000000e+20,5.000000e+41,0.0,0.0,476.526902,2.265564e+07,14.788297,1.255135e+04
2,0.0,8.600351e+06,8.370657,14642.714232,5.000000e+20,5.000000e+41,0.0,0.0,-4.379829,2.165183e+03,0.791440,1.831766e+03
3,0.0,8.600351e+06,-0.302083,314.394731,5.000000e+20,5.000000e+41,0.0,0.0,-0.424561,5.120240e+02,-4.694587,1.217540e+03
4,0.0,8.600351e+06,-8.137312,10466.388533,5.000000e+20,5.000000e+41,0.0,0.0,-1.820296,6.630281e+02,1.095076,2.545859e+02
...,...,...,...,...,...,...,...,...,...,...,...,...
95,0.0,8.600351e+06,-4.166445,1554.349765,5.000000e+20,5.000000e+41,0.0,0.0,-8.198049,5.043250e+03,0.126027,1.600725e+02
96,0.0,8.600351e+06,-19.907987,22353.229761,5.000000e+20,5.000000e+41,0.0,0.0,3.828542,4.220748e+03,-184.324003,3.527710e+06
97,0.0,8.600351e+06,-2.656682,909.913591,5.000000e+20,5.000000e+41,0.0,0.0,-0.776851,3.500071e+02,0.043850,4.853376e+02
98,0.0,8.600351e+06,-36.054546,127228.386620,5.000000e+20,5.000000e+41,0.0,0.0,-1.070722,1.238637e+02,-7.898211,8.057112e+03


In [ ]:
Delta

1e-46